# Các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from time import sleep

In [2]:
# khai báo driver
driver = webdriver.Chrome()

In [3]:
# mở URL
driver.get("https://www.lazada.vn/dien-thoai-di-dong/?spm=a2o4n.home-vn.6598063730.37.19053bdcNwANax&up_id=2301126560&clickTrackInfo=491be505-9e8e-45ac-bbf5-f57141188de1__4518__2301126560__20__0.0989122__341738__7253__c2i__0.0__53%2525%2Boff&from=hp_categories&item_id=2301126560&version=v2&q=Mobiles&params=%7B%22catIdLv1%22%3A%224402%22%2C%22pvid%22%3A%22491be505-9e8e-45ac-bbf5-f57141188de1%22%2C%22src%22%3A%22ald%22%2C%22categoryName%22%3A%22Mobiles%22%2C%22categoryId%22%3A%224518%22%7D&src=hp_categories")
driver.implicitly_wait(10)

In [4]:
elements_title = driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")
lst_Title = [element_title.text for element_title in elements_title]
lst_Link = [element_title.get_attribute('href') for element_title in elements_title]

In [5]:
elements_discPrice = driver.find_elements(By.CSS_SELECTOR, ".aBrP0")
lst_DiscPrice = [element_discPrice.text for element_discPrice in elements_discPrice]

In [6]:
elements_valDiscount = driver.find_elements(By.CSS_SELECTOR, ".IcOsH")
lst_ValDiscount = [element_valDiscount.text for element_valDiscount in elements_valDiscount]

In [7]:
lst_ValDiscount = [valDiscount.replace(' Off', '') for valDiscount in lst_ValDiscount]

In [8]:
elements_sold_countReview_sellAt = driver.find_elements(By.CSS_SELECTOR, "._6uN7R")
lst_Sold_countReview_sellAt = [element_sold_countReview_sellAt.text for element_sold_countReview_sellAt in elements_sold_countReview_sellAt]

In [9]:
def handle_arr_sold_countReview_sellAt(data_arr):
    sold = []
    countReview = []
    sellAt = []

    for data in data_arr:
        parts = data.split()
        soldCount = None
        reviewCount = None
        location = None

        if "Đã bán" in parts:
            try:
                soldCount = int(parts[0])
            except ValueError:
                soldCount = None
            
            try:
                reviewIndex = parts.index("Đã bán") + 1
                reviewCount = int(parts[reviewIndex].strip("()"))
            except (ValueError, IndexError):
                reviewCount = None

            location = parts[reviewIndex + 1:]
        else:
            location = data
        
        sold.append(soldCount)
        countReview.append(reviewCount)
        sellAt.append(location)

    return sold, countReview, sellAt


In [10]:
lst_Sold, lst_CountReview, lst_SellAt = handle_arr_sold_countReview_sellAt(lst_Sold_countReview_sellAt)

In [13]:
df = pd.DataFrame(list(zip(lst_Title, lst_DiscPrice, lst_ValDiscount, lst_Sold, lst_CountReview, lst_SellAt, lst_Link)), 
                  columns = ['title', 'discount_Price', 'value_Discount', 'sold', 'count_Review', 'sell_At', 'link_Item'])

df['index_'] = np.arange(1, len(df) + 1)
df = df[['index_', 'title', 'discount_Price', 'value_Discount', 'sold', 'count_Review', 'sell_At', 'link_Item']]

In [14]:
df.to_csv('data_crawl_selenium_list.csv', index = False)